
## Ионов Тимур ИУ9-71Б
### Лабораторная работа N6 «Изучение сходимости метода Якоби»

### Цель работы
1. Реализовать метод Якоби.
2. Ввести критерий остановки итерационного процесс используя равномерную норму. 3. Проверить решение путем сравнения с решением любым методом Гаусса.
4. Проверить выполнение условия диагонального преобладания.
5. Используя согласованную векторную и матричную нормы проверить выполнение условиe ||P|| < q < 1

In [1]:
import numpy as np
from copy import deepcopy

In [2]:
def gen_matrix(n, a=-1, b=1, triag_dom=1):
    x = np.random.uniform(a, b, size=(n, n))
    if triag_dom > 1:
        ids = np.diag_indices(n)
        diag = np.sum(np.abs(x), axis=1)
        diag *= triag_dom
        x[ids] = diag
    return x

In [3]:
def gauss(a, b, row_opt=False, col_opt=False, both_opt=False):
    a = deepcopy(a)
    b = deepcopy(b)
    n = len(b)
    
    assert len(a.shape) == 2 and a.shape[0] == a.shape[1]
    assert not any(np.diag(a) == 0)
    
    # forward
    order = np.arange(n)
    for k in range(n-1):
        for i in range(k + 1, n):
            if both_opt:
                row_max = np.max(np.abs(a[k:, k]))
                col_max = np.max(np.abs(a[k, k:]))
                if row_max > col_max:
                    row_opt = True
                    col_opt = False
                else:
                    row_opt = False
                    col_opt = True
            if row_opt:
                ind = k + np.argmax(np.abs(a[k:, k]))
                a[[k, ind]] = a[[ind, k]]
                b[ind], b[k] = b[k], b[ind]
            elif col_opt:
                ind = k + np.argmax(np.abs(a[k, k:]))
                order[k], order[ind] = order[ind], order[k]
                a[:,[k, ind]] = a[:,[ind, k]]
                
            # elimination
            r = a[i, k] / a[k, k]
            a[i, :] -= a[k, :] * r
            b[i] -= b[k] * r
    
    
    # backward
    unordered_x = np.zeros(n)    
    for i in range(n-1, -1, -1):
        unordered_x[i] = (b[i] - np.dot(a[i, i:n], unordered_x[i:n])) / a[i, i]
    x = np.zeros(n)
    for i, o in enumerate(order):
        x[o] = unordered_x[i]
    return x

In [4]:
def vec_norm(x):
    return max(abs(x))

def mat_norm(A):
    return max(np.sum(np.abs(A), axis=1))

In [5]:
def check_diag_dominance(a):
    n = a.shape[0]
    for i in range(n):
        if a[i][i] < sum(a[i][[j for j in range(n) if j != i]]):
            return False
    return True

In [27]:
def jacobi(A, b, eps=1e-10):
    assert check_diag_dominance(A), 'A isnt triag dominance'
                                                                                                                                     
    x = np.zeros_like(b)
    
    D = np.diag(A)
    R = (A - np.diagflat(D)) / D
    g = b / D
    
    assert mat_norm(R) < 1, 'R matrix has norm > 1'
    
    c = 0
    while True:
        c += 1
        prev_x = x
        x = g - np.dot(R, x)
        
        if vec_norm(prev_x - x) < eps:
            return x, mat_norm(R), c
        

In [28]:
N = 4
LOW = -10
HIGH = 10
A = gen_matrix(N, a=LOW, b=HIGH, triag_dom=3)
b = np.random.uniform(LOW, HIGH, N)

In [29]:
x_gauss = gauss(A, b)

In [30]:
x_gauss

array([-0.01487189, -0.08072054, -0.15713376,  0.00617757])

In [31]:
x_jacobi, n, c = jacobi(A, b)

In [32]:
c

13

In [25]:
x_jacobi

array([ 0.08049633,  0.01904396,  0.05104557, -0.05123509])

In [12]:
vec_norm(x_jacobi-x_gauss)

0.03781211363191486

In [13]:
!jupyter-nbconvert --to PDFviaHTML lab6.ipynb

[NbConvertApp] Converting notebook lab6.ipynb to PDFviaHTML
[NbConvertApp] Writing 416620 bytes to lab6.pdf
